In [1]:
pip install --upgrade setuptools wheel --user -i https://mirrors.aliyun.com/pypi/simple/

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install biopython --user -i https://mirrors.aliyun.com/pypi/simple/

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
sys.path.append('./site-packages')

In [6]:
pip install transformers sentencepiece h5py --user -i https://mirrors.aliyun.com/pypi/simple/

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pytz --user -i https://mirrors.aliyun.com/pypi/simple/

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pandas scikit-learn matplotlib --user -i https://mirrors.aliyun.com/pypi/simple/

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install evaluate datasets tqdm scipy --user -i https://mirrors.aliyun.com/pypi/simple/

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os.path
os.chdir("./ProtT5_model")

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from torch.utils.data import DataLoader

import re
import numpy as np
import pandas as pd
import copy

import transformers, datasets
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.t5.modeling_t5 import T5Config, T5PreTrainedModel, T5Stack
from transformers.utils.model_parallel_utils import assert_device_map, get_device_map
from transformers import T5EncoderModel, T5Tokenizer
from transformers import TrainingArguments, Trainer, set_seed

from evaluate import load
from datasets import Dataset

from tqdm import tqdm
import random
from scipy import stats
from sklearn.metrics import accuracy_score
import torch
import h5py
import time
import matplotlib.pyplot as plt
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')###推理任务仅需单块GPU
print("Using {}".format(device))

/share/home/yzwl_sunjw/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda:0


In [2]:
seq_path ='./DC.fasta'
#仅使用per_protein读取嵌入
per_protein=True
per_protein_path = "./DC.h5"
per_residue=False
sec_struct=False
assert per_protein is True or per_residue is True or sec_struct is True, print(
    "False")

In [3]:
###这里的bin用的是全精度
def get_T5_model():
    model = T5EncoderModel.from_pretrained("./ProtT5_model")#.bin权重文件本地绝对路径
    model = model.to(device) 
    model = model.eval() 
    tokenizer = T5Tokenizer.from_pretrained('/share/home/yzwl_sunjw/ProtT5_model', do_lower_case=False)#分词器本地绝对路径

    return model, tokenizer

In [9]:
###读取fasta文件
def read_fasta( fasta_path, split_char="!", id_field=0):
    '''
        Reads in fasta file containing multiple sequences.
        Split_char and id_field allow to control identifier extraction from header.
        E.g.: set split_char="|" and id_field=1 for SwissProt/UniProt Headers.
        Returns dictionary holding multiple sequences or only single
        sequence, depending on input file.
    '''

    seqs = dict()
    with open( fasta_path, 'r' ) as fasta_f:
        for line in fasta_f:
            # get uniprot ID from header and create new entry
            if line.startswith('>'):
                uniprot_id = line.replace('>', '').strip().split(split_char)[id_field]
                # replace tokens that are mis-interpreted when loading h5
                uniprot_id = uniprot_id.replace("/","_").replace(".","_")
                seqs[ uniprot_id ] = ''
            else:
                # repl. all whie-space chars and join seqs spanning multiple lines, drop gaps and cast to upper-case
                seq= ''.join( line.split() ).upper().replace("-","")
                # repl. all non-standard AAs and map them to unknown/X
                seq = seq.replace('U','X').replace('Z','X').replace('O','X')
                seqs[ uniprot_id ] += seq
    example_id=next(iter(seqs))
    print("Read {} sequences.".format(len(seqs)))
    print("Example:\n{}\n{}".format(example_id,seqs[example_id]))

    return seqs

In [10]:
#获取embeddings并返回
def get_embeddings( model, tokenizer, seqs, per_residue, per_protein, sec_struct,
                   max_residues=9000, max_seq_len=1000, max_batch=100 ):

    if sec_struct:
      sec_struct_model = load_sec_struct_model()

    results = {"residue_embs" : dict(),
               "protein_embs" : dict(),
               "sec_structs" : dict()
               }
    seq_dict   = sorted( seqs.items(), key=lambda kv: len( seqs[kv[0]] ), reverse=True )
    start = time.time()
    batch = list()
    for seq_idx, (pdb_id, seq) in enumerate(seq_dict,1):
        seq = seq
        seq_len = len(seq)
        seq = ' '.join(list(seq))
        batch.append((pdb_id,seq,seq_len))
        n_res_batch = sum([ s_len for  _, _, s_len in batch ]) + seq_len
        if len(batch) >= max_batch or n_res_batch>=max_residues or seq_idx==len(seq_dict) or seq_len>max_seq_len:
            pdb_ids, seqs, seq_lens = zip(*batch)
            batch = list()
            token_encoding = tokenizer.batch_encode_plus(seqs, add_special_tokens=True, padding="longest")
            input_ids      = torch.tensor(token_encoding['input_ids']).to(device)
            attention_mask = torch.tensor(token_encoding['attention_mask']).to(device)

            try:
                with torch.no_grad():
                    # returns: ( batch-size x max_seq_len_in_minibatch x embedding_dim )
                    embedding_repr = model(input_ids, attention_mask=attention_mask)
            except RuntimeError:
                print("RuntimeError during embedding for {} (L={})".format(pdb_id, seq_len))
                continue

            if sec_struct: 
              d3_Yhat, d8_Yhat, diso_Yhat = sec_struct_model(embedding_repr.last_hidden_state)


            for batch_idx, identifier in enumerate(pdb_ids): 
                s_len = seq_lens[batch_idx]
                emb = embedding_repr.last_hidden_state[batch_idx,:s_len]
                if sec_struct: 
                    results["sec_structs"][identifier] = torch.max( d3_Yhat[batch_idx,:s_len], dim=1 )[1].detach().cpu().numpy().squeeze()
                if per_residue: 
                    results["residue_embs"][ identifier ] = emb.detach().cpu().numpy().squeeze()
                if per_protein: 
                    protein_emb = emb.mean(dim=0)
                    results["protein_embs"][identifier] = protein_emb.detach().cpu().numpy().squeeze()


    passed_time=time.time()-start
    avg_time = passed_time/len(results["residue_embs"]) if per_residue else passed_time/len(results["protein_embs"])
    print('\n############# EMBEDDING STATS #############')
    print('Total number of per-residue embeddings: {}'.format(len(results["residue_embs"])))
    print('Total number of per-protein embeddings: {}'.format(len(results["protein_embs"])))
    print("Time for generating embeddings: {:.1f}[m] ({:.3f}[s/protein])".format(
        passed_time/60, avg_time ))
    print('\n############# END #############')
    return results

In [11]:
def save_embeddings(emb_dict,out_path):
    with h5py.File(str(out_path), "w") as hf:
        for sequence_id, embedding in emb_dict.items():
            hf.create_dataset(sequence_id, data=embedding)
    return None

In [13]:
model, tokenizer = get_T5_model()

seqs = read_fasta( seq_path )

results = get_embeddings( model, tokenizer, seqs,
                         per_residue, per_protein, sec_struct)

if per_residue:
  save_embeddings(results["residue_embs"], per_residue_path)
if per_protein:
  save_embeddings(results["protein_embs"], per_protein_path)
if sec_struct:
  write_prediction_fasta(results["sec_structs"], sec_struct_path)

Read 1 sequences.
Example:
tr|Q87W42|Q87W42_PSESM Type III effector HopG1 OS=Pseudomonas syringae pv_ tomato (strain ATCC BAA-871 _ DC3000) OX=223283 GN=hopG1 PE=4 SV=1
MQIKNSHLYSASRMVQNTFNASPKMEVTNAIAKNNEPAALSATQTAKTHEGDSKGQSSNNSKLPFRAMRYAAYLAGSAYLYDKTANNFFLSTTSLHDGKGGFTSDARLNDAQDKARKRYQNNHSSTLENKNSLLSPLRLCGENQFLTMIDYRAATKIYLSDLVDTEQAHTSILKNIMCLKGELTNEEAIKKLNPEKTPKDYDLTNSEAYISKNKYSLTGVKNEETGSTGYTSRSITKPFVEKGLKHFIKATHGEKALTPKQCMETLDNLLRKSITLNSDSQFAAGQALLVFRQVYAGEDAWGDAERVILKSHYNRGTVLQDEADKIELSRPFSEQDLAKNMFKRNTSIAGPVLYHAYIYIQEKIFKLPPDKIEDLKHKSMADLKNLPLTHVKLSNSGVGFEDASGLGDSFTALNATSCVNHARIMSGEPPLSKDDVVILIGCLNAVYDNSSGIRHSLREIARGCFVGAGFTVQDGDDFYKQICKNASKQFYNG

############# EMBEDDING STATS #############
Total number of per-residue embeddings: 0
Total number of per-protein embeddings: 1
Time for generating embeddings: 0.0[m] (2.980[s/protein])

############# END #############
